# Writing data to and reading data from a Database using Python

In [9]:
import os
import sqlite3
from sqlalchemy import create_engine, text
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import psycopg2

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# Get current working directory
print(os.getcwd())

/workspaces/remo-caprice-project/Database


# Create a Database

In [10]:
# Set up database connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "postgres"

# Erstellen der Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test database connection
with engine.connect() as connection:
    result = connection.execute(text('SELECT current_database()'))
    print(result.fetchone())

# Dispose the engine
engine.dispose()

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# --- Configuration ---
DATABASE_FILE = 'zuerich_properties.db'
TABLE_NAME = 'properties_zuerich'
CSV_FILE = 'properties_data_prepared_zuerich.csv'

# 1. Load the final cleaned DataFrame
df = pd.read_csv(CSV_FILE, encoding='utf-8')[['web_scraper_order','address_raw', 'rooms', 'area', 'rent']]

# 2. Connect to the SQLite Database
# This creates the file if it doesn't exist
try:
    conn = sqlite3.connect(DATABASE_FILE)
    print(f"✅ SQLite database created/connected: {DATABASE_FILE}")

    # 3. Upload Data to the Table
    # The 'to_sql' method is a powerful part of Pandas that handles schema creation
    # and data insertion efficiently.
    df.to_sql(
        TABLE_NAME, 
        conn, 
        if_exists='replace', # Replace table if it already exists
        index=False          # Do not upload the pandas index
    )

    print(f"\n✅ Success! Data from {len(df)} rows has been loaded into the '{TABLE_NAME}' table.")

except Exception as e:
    print(f"\n❌ Error connecting to or writing to the database:")
    print(e)

# Create cursor object to execute SQL commands
cursor = conn.cursor()
        
df.head()

## Write data to the SQL-table in data base

In [ ]:
df.to_sql(name = 'properties_zuerich',
          con = conn,
          index = False,
          if_exists = 'replace')

## Query the SQL-table

In [ ]:
# Define the SQL Query
cursor.execute('''SELECT *
               FROM properties_zuerich
               WHERE rooms >=1''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['web_scraper_order','Address_raw','Rooms','Area','Rent'])    
df

### safe the new data in a csv file

In [ ]:
# safe the new data in a csv file
df.to_csv('properties_data_final_zuerich.csv', index=False)



## Plot histogramm of rental prices

In [ ]:
# Create a histogram
plt.figure(figsize=(7,4))
df.Rent.plot.hist(grid=True, 
                   bins=20, 
                   rwidth=0.9,
                   color='#607c8e')
plt.title('Apartment Price distribution in Zurich')
plt.xlabel('Price (CHF)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)

## Close db connection (if open)

In [ ]:
# Close db connection (if open)
try:
    if check_conn(conn):
        conn.close()
    else:
        pass
except:
    pass

# Status (True = open, False = closed)
print(check_conn(conn))

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')